<a href="https://colab.research.google.com/github/muhammad-usama-aleem/classification_general_military_tank/blob/main/modified_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!pip install unrar
!unrar x /content/gdrive/MyDrive/dataset.rar

In [3]:
import os

base_dir = '/content/combined/'
os.mkdir(base_dir)

!mv /content/tanks /content/combined
!mv /content/men /content/combined
!mv /content/army /content/combined

In [4]:
# removing gif file

import os

# !rm -rf /content/main
img = os.listdir('/content/combined/men')
for i,v in enumerate(img):
  if v[-3:] == 'gif':
    # !rm -rf /content/combined/men/img[i]
    os.remove("/content/combined/men/{}".format(img[i]))

In [5]:
import os
for File in os.listdir("/content/combined/men"):
    if File.endswith(".gif"):
        print("true")

In [6]:
import os

main_dir = '/content/main/'
os.mkdir(main_dir)

In [7]:
# !rm -rf /content/main

In [7]:
#create two folders (train and validation)
train_dir = os.path.join(main_dir, 'train')
os.mkdir(train_dir)

test_dir = os.path.join(main_dir, 'test')
os.mkdir(test_dir)

#Under train folder create five folders 

train_cloud_dir = os.path.join(train_dir, 'army')
os.mkdir(train_cloud_dir)

train_foggy_dir = os.path.join(train_dir, 'men')
os.mkdir(train_foggy_dir)

train_rainy_dir = os.path.join(train_dir, 'tanks')
os.mkdir(train_rainy_dir)

#Under validation folder create five folders 

validation_cloud_dir = os.path.join(test_dir, 'army')
os.mkdir(validation_cloud_dir)

validation_foggy_dir = os.path.join(test_dir, 'men')
os.mkdir(validation_foggy_dir)

validation_rainy_dir = os.path.join(test_dir, 'tanks')
os.mkdir(validation_rainy_dir)

In [8]:
army_SOURCE_DIR = '/content/combined/army/'
TRAINING_army_DIR = '/content/main/train/army/'
test_army_DIR = '/content/main/test/army/'

men_SOURCE_DIR = '/content/combined/men/'
TRAINING_men_DIR = '/content/main/train/men/'
test_men_DIR = '/content/main/test/men/'

tanks_SOURCE_DIR = '/content/combined/tanks/'
TRAINING_tanks_DIR = '/content/main/train/tanks/'
test_tanks_DIR = '/content/main/test/tanks/'

In [9]:
def split_data(SOURCE, TRAINING, VALIDATION, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")

    training_length = int(len(files) * SPLIT_SIZE)
    print('training_length', training_length)
    valid_length = int(len(files) - training_length)
    print('test_length', valid_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    valid_set = shuffled_set[training_length:]

    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file, destination)

    for filename in valid_set:
        this_file = SOURCE + filename
        destination = VALIDATION + filename
        copyfile(this_file, destination)

In [10]:
import os
import random
from shutil import copyfile

split_size = .85

split_data(army_SOURCE_DIR, TRAINING_army_DIR, test_army_DIR, split_size)
split_data(men_SOURCE_DIR, TRAINING_men_DIR, test_men_DIR, split_size)
split_data(tanks_SOURCE_DIR, TRAINING_tanks_DIR, test_tanks_DIR, split_size)

training_length 1275
test_length 225
training_length 1197
test_length 212
training_length 917
test_length 162


# train data

In [11]:
import numpy as np
import tensorflow as tf

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('/content/main/train')

print (dataset_path)  #what kinds of classes are in this dataset

print("Types of classes labels found: ", len(dataset_path))

['tanks', 'army', 'men']
Types of classes labels found:  3


In [23]:
train_class_labels = []

for item in dataset_path:
 # Get all the file names
 all_classes = os.listdir('/content/main/train' + '/' +item)
#  print(all_classes)

 # Add them to the list
 for room in all_classes:
    train_class_labels.append((item, str('dataset_path' + '/' +item) + '/' + room))
    # print(dataset_path, item, room)

In [24]:
# Build a dataframe        
df = pd.DataFrame(data=train_class_labels, columns=['Labels', 'image'])
print(df.head())
print(df.tail())

  Labels                                  image
0  tanks          dataset_path/tanks/t55_l1.jpg
1  tanks      dataset_path/tanks/tank_ex_l2.jpg
2  tanks       dataset_path/tanks/jaguar_l2.jpg
3  tanks       dataset_path/tanks/bmpt_main.jpg
4  tanks  dataset_path/tanks/leopard_2ng_l1.jpg
     Labels                          image
3384    men  dataset_path/men/00001100.jpg
3385    men  dataset_path/men/00001017.jpg
3386    men  dataset_path/men/00002299.jpg
3387    men  dataset_path/men/00000135.jpg
3388    men  dataset_path/men/00001706.jpg


In [25]:
# Let's check how many samples for each category are present
print("Total number of images in the dataset: ", len(df))

label_count = df['Labels'].value_counts()
print(label_count)

Total number of images in the dataset:  3389
army     1275
men      1197
tanks     917
Name: Labels, dtype: int64


In [26]:
import cv2
path = '/content/main/train/'
dataset_path = os.listdir('/content/main/train')

im_size = 224

train_images = []
train_labels = []

for i in dataset_path:
    data_path = path + str(i)
    filenames = [i for i in os.listdir(data_path) ]
    print(data_path)
    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.resize(img, (im_size, im_size))
        train_images.append(img)
        train_labels.append(i)

/content/main/train/tanks
/content/main/train/army
/content/main/train/men


In [27]:
train_images = np.array(train_images)
print(train_images.shape)
train_images = train_images.astype('float32') / 255.0
# train_images = preprocess_input(train_images)
print(train_images.shape)

(3389, 224, 224, 3)
(3389, 224, 224, 3)


In [28]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
y=df['Labels'].values
print(y)
print(len(y))
print(list(set(y)))
y_labelencoder = LabelEncoder ()
y = y_labelencoder.fit_transform (y)
print(y)
print(list(set(y)))

['tanks' 'tanks' 'tanks' ... 'men' 'men' 'men']
3389
['tanks', 'army', 'men']
[2 2 2 ... 1 1 1]
[0, 1, 2]


In [29]:
y=y.reshape(-1,1)

from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([('my_ohe', OneHotEncoder(), [0])], remainder='passthrough')
Y = ct.fit_transform(y) #.toarray()
print(Y[:5])
print(Y[:-5])

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [30]:
height = 224
width = 224
channels = 3

n_classes = 3
input_shape = (height, width, channels)

epochs = 50
batch_size = 16

In [31]:
from tensorflow.keras.applications import EfficientNetB0 as Net
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras import layers


NUM_CLASSES = 3
IMG_SIZE = 224
size = (IMG_SIZE, IMG_SIZE)


inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

# loading pretrained conv base model
outputs = Net(weights=None, include_top=True, classes=NUM_CLASSES)(inputs)

In [32]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers


train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
)

train_dir = '/content/main/train'
# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    # This is the target directory
    train_dir,
    # All images will be resized to target height and width.
    target_size=(height, width),
    batch_size=batch_size,
    # Since we use categorical_crossentropy loss, we need categorical labels
    class_mode="categorical",
)

validation_dir = '/content/main/test'

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode="categorical",
)

model = tf.keras.Model(inputs, outputs)

model.compile(
    loss="categorical_crossentropy",
    optimizer=optimizers.RMSprop(learning_rate=2e-5),
    metrics=["accuracy"],
)
model.summary()

Found 3389 images belonging to 3 classes.
Found 599 images belonging to 3 classes.
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, 3)                 4053414   
Total params: 4,053,414
Trainable params: 4,011,391
Non-trainable params: 42,023
_________________________________________________________________


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    verbose=1,
    use_multiprocessing=True,
    workers=4,
)

In [24]:
model.save_weights("classification_efficientnetb0_weights.h5")

In [25]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
from google.colab import files
files.download("/content/classification_efficientnetb0_weights.h5")

files.download("/content/model.json")

In [27]:
CLASSES = ['army', 'men', 'tanks']

In [28]:
validation_generator.batch_size=1
score = model.evaluate_generator(validation_generator, validation_generator.samples//4)

# test_data_gen.reset()
print('\n')
print("Loss: ", score[0], '\n',"Accuracy: ", score[1])

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2006: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '




Loss:  0.6790211200714111 
 Accuracy:  0.7718120813369751


In [29]:
scores=model.predict(validation_generator)
print("Loss: ", score[0], "Accuracy: ", score[1])

predicted_class_indices=np.argmax(scores,axis=1)
print(predicted_class_indices)


Loss:  0.6790211200714111 Accuracy:  0.7718120813369751
[0 1 1 0 0 0 2 0 0 2 1 2 0 0 0 0 2 2 2 0 2 0 0 2 2 2 0 0 1 2 1 0 1 1 0 0 1
 1 1 0 2 2 2 1 0 0 1 1 2 1 0 0 0 0 0 0 0 1 2 1 2 2 0 2 0 1 2 0 1 0 0 2 0 0
 0 2 1 0 1 0 0 0 1 2 0 0 2 1 1 1 2 0 2 1 1 0 2 1 1 2 1 2 0 0 0 2 2 0 1 2 1
 0 1 0 0 2 1 2 0 2 1 0 2 0 0 0 0 0 0 1 2 0 2 1 2 2 2 1 2 0 0 2 2 2 2 0 1 0
 1 2 1 2 0 2 2 0 1 0 1 0 0 0 1 0 2 2 1 2 1 1 0 2 2 2 2 2 1 0 1 0 1 1 0 0 1
 2 0 0 2 1 2 0 1 2 2 1 2 0 0 2 1 1 2 0 2 0 0 1 0 1 0 2 0 0 1 2 1 2 0 1 0 2
 2 2 2 1 0 0 2 0 0 0 2 1 2 0 0 2 2 1 1 0 0 0 2 2 1 0 1 0 1 1 1 2 0 2 1 0 0
 2 1 0 2 2 2 1 2 2 1 0 2 2 2 1 0 1 1 0 2 0 2 1 2 0 2 2 2 1 2 1 2 0 1 2 1 0
 1 0 2 0 0 1 0 1 2 2 2 2 2 1 2 1 0 2 1 0 2 2 2 1 0 2 0 2 1 0 0 0 1 0 2 0 1
 0 0 1 1 2 2 1 0 2 1 1 1 1 2 2 0 2 1 0 0 1 0 2 0 0 0 0 1 1 0 1 0 0 2 1 0 0
 2 1 0 2 1 1 2 1 2 2 0 1 0 2 1 1 0 0 2 2 1 1 2 2 1 2 1 0 1 2 2 1 1 2 2 2 1
 1 1 2 0 2 2 1 2 2 2 1 1 0 0 2 0 0 0 2 0 2 1 1 1 1 1 2 2 0 2 2 1 0 0 2 2 1
 1 2 0 0 1 0 0 1 0 0 2 2 1 0 2 0 2 2 0 0 0 1

In [30]:
labels = (validation_generator.class_indices)
labelss = dict((v,k) for k,v in labels.items())
predictions = [labelss[k] for k in predicted_class_indices]

filenames=validation_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})

print(results)

                      Filename Predictions
0                army/1003.png        army
1                 army/101.png         men
2                army/1024.png         men
3                army/1043.png        army
4                army/1046.png        army
..                         ...         ...
594    tanks/zulfiqar_1_l3.jpg       tanks
595    tanks/zulfiqar_1_l4.jpg         men
596    tanks/zulfiqar_1_l7.jpg         men
597    tanks/zulfiqar_2_l1.jpg        army
598  tanks/zulfiqar_3_main.jpg       tanks

[599 rows x 2 columns]


In [ ]:
results.to_csv('results.csv')

In [ ]:
files.download("/content/results.csv")

In [12]:
import os

res_dir = '/content/results/'
os.mkdir(res_dir)

#create two folders (train and validation)
army_dir = os.path.join(res_dir, 'army')
os.mkdir(army_dir)

men_dir = os.path.join(res_dir, 'men')
os.mkdir(men_dir)

tanks_dir = os.path.join(res_dir, 'tanks')
os.mkdir(tanks_dir)

#Under army folder create five folders 

army_cloud_dir = os.path.join(army_dir, 'army')
os.mkdir(army_cloud_dir)

army_foggy_dir = os.path.join(army_dir, 'men')
os.mkdir(army_foggy_dir)

army_rainy_dir = os.path.join(army_dir, 'tanks')
os.mkdir(army_rainy_dir)

#Under men folder create five folders 

men_cloud_dir = os.path.join(men_dir, 'army')
os.mkdir(men_cloud_dir)

men_foggy_dir = os.path.join(men_dir, 'men')
os.mkdir(men_foggy_dir)

men_rainy_dir = os.path.join(men_dir, 'tanks')
os.mkdir(men_rainy_dir)

#Under tanks folder create five folders 

tanks_cloud_dir = os.path.join(tanks_dir, 'army')
os.mkdir(tanks_cloud_dir)

tanks_foggy_dir = os.path.join(tanks_dir, 'men')
os.mkdir(tanks_foggy_dir)

tanks_rainy_dir = os.path.join(tanks_dir, 'tanks')
os.mkdir(tanks_rainy_dir)

In [13]:
import pandas as pd

final = pd.read_csv('results.csv')
final.head()

,Unnamed: 0,Filename,Predictions
0,0,army/1003.png,army
1,1,army/101.png,men
2,2,army/1024.png,men
3,3,army/1043.png,army
4,4,army/1046.png,army


In [14]:
for num in range(599):
  img_path = '/content/combined/' + final.Filename[num]
  des = '/content/results/' + final.Predictions[num] + '/' + final.Filename[num]
  copyfile(img_path, des)

In [ ]:
!zip -r /content/result.zip /content/results

In [16]:
from google.colab import files
files.download("/content/result.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>